# LidarDM's KITTI360 Generation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzyrianov/bev-ldg/blob/main/examples/notebook_kitti.ipynb)

#### Colab setup

In [ ]:
!git clone https://github.com/vzyrianov/bev-ldg.git
%cd bev-ldg

# install packages
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install hydra-core
!pip install lightning
!pip install tensorboard
!pip install wandb
!pip install transformers
!pip install matplotlib
!pip install natsort
!pip install pykdtree
!pip install pyntcloud
!pip install pyquaternion
!pip install scikit-learn
!pip install scipy
!pip install tqdm
!pip install open3d
!pip install opencv-python
!pip install diffusers==0.20.2
!pip install nksr==1.0.3+pt20cu117           -f https://nksr.huangjh.tech/whl/torch-2.0.0+cu117.html
!pip install torch-scatter==2.1.2+pt20cu117  -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torchmetrics==1.2.0
!pip install triton==2.0.0
!pip install -v -e . # for lidardm

# download small asset bank
!wget -O assets.tar.xz "https://uofi.box.com/shared/static/ir42y4a71luia12u17bk5v0fierqrogw.xz" 
!tar -xvf assets.tar.xz

# download weights 
!mkdir pretrained_models
%cd pretrained_models
!wget -O kitti_weights.zip "https://uofi.box.com/shared/static/tc4hppt38ryy5rsgthiw4q50dxtu1w2f.zip" 
!unzip kitti_weights.zip

#### Env setup

In [1]:
import open3d as o3d
import numpy as np

import mediapy
from utils import get_visualization_videos

from lidardm.core.services.sampling import sample_uncond_kitti, instantiate_unconditional_model_kitti
from lidardm.lidar_generation.scene_composition.compositors import GenerativeCompositor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2024-03-19 21:17:59.428951: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hche/miniconda3/envs/waymo/lib/python3.10/site-packages/cv2/../../lib64:
2024-03-19 21:17:59.429001: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hche/miniconda3/envs/waymo/lib/python3.10/site-packages/cv2/../../lib64:
2024-03-19 21:17:59.429004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
03-19 21:18:00 (o3d.py:20) [WARNING] Customiz

#### Generating the static world

In [2]:
model = instantiate_unconditional_model_kitti("../pretrained_models/kitti360/scene_gen/kfsvae_kl1e-7.ckpt", 
                                              "../pretrained_models/kitti360/scene_gen/kfsunet_b.ckpt")
model = model.cuda()
mesh = sample_uncond_kitti(model)

/home/hche/miniconda3/envs/waymo/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


#### Composing underlying 4D scene

If you are running this notebook via SSH, it is likely that Open3D headless rendering won't work. set `USING_SSH = True` for (not-as-pretty-but-working) visualization

In [3]:
USING_SSH = False

compositor = GenerativeCompositor(mesh=mesh, n_frames=20)
videos = get_visualization_videos(compositor, USING_SSH)
mediapy.show_videos(videos, fps=10, height=400)

Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Waymo Dataset isn't found, use straight ego-poses
False
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0xe4960e10 (threading is enabled)
EGL(1.5)
OpenGL(4.6)


Grabbing viz: 100%|██████████| 20/20 [00:09<00:00,  2.22it/s]


This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.
